In [1]:
### Import libraries

import igraph as ig
import numpy as np
import os
from collections import Counter, OrderedDict
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from auxiliar_bb import apply_backboning
from auxiliar_projections_large import apply_projection
from auxiliar_opti_two import *
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from matplotlib.lines import Line2D

In [19]:
### Select dataset
DATASET = "am" # ["amz", "um", "am"]
PROJ_NAMES = ["simple", "vector", "jaccard", "neighs", "hyper", "resall"]

nombre_proyecciones = {
    "simple": "Weighted", "jaccard": "Jaccard", "hyper": "Hyperbolic",
    "vector": "Vectorized", "resall": "Resource\nAllocation", "neighs": "Neighborhood"
}
nombre_backbonings = { "NC": "Noise Corrected", "DF": "Disparity Filter"}
COLORS_PR = ["#ef476fff", "#f78c6bff", "#ffd166ff", "#06d6a0ff", "#118ab2ff", "#718355"]
COLORS_BB = [ "#8ab17d", "#e9c46a"]
COLORS_SV = ["#718355", "#87986a", "#97a97c", "#b5c99a"]

if DATASET == "um":
    FILENAME = "../12-third_year/00-Data/08-imdb/user-movie.graphml"
elif DATASET == "am":
    FILENAME = "/home/daniel/Documents/phd/phd-thesis-lab/12-third_year/00-Data/05-actor-movie/actor-movie.graphml"
elif DATASET == "amz":
    FILENAME = "binet-AMZ-Rw.graphml"


In [20]:
g = ig.read(FILENAME)
print(g.summary())
print()

top_nodes = g.vs.select(type=0)
bot_nodes = g.vs.select(type=1)

print("|U|=",len(top_nodes), " \t|R|=",len(bot_nodes), " \t|U|+|R|=",
      len(top_nodes)+len(bot_nodes), "=", g.vcount())
print()

if(g.is_bipartite()): # Check if the the graph is bipartite
    print("The graph IS bipartite")
else:
    print("The graph IS NOT bipartite")
    exit()


IGRAPH UN-T 511463 1470404 -- 
+ attr: id (v), name (v), type (v)

|U|= 383640  	|R|= 127823  	|U|+|R|= 511463 = 511463

The graph IS bipartite


In [4]:
avg_cc_bip = 0
for i in range(5):
    res = average_local_bipartite_clustering_coefficient_sampled_stratified(g, n=500)
    avg_cc_bip += res
avg_cc_bip / 5

KeyboardInterrupt: 

In [28]:
average_local_bipartite_clustering_coefficient_sampled_stratified(g, n=100)

0.32535131844540927

In [21]:
compute_power_law_bipartite(g, 0)

1.8623650345000595

In [22]:
compute_power_law_bipartite(g, 1)

5.066327861107771

In [23]:
compute_avg_path_length_same_type(g, k=100, seed=42)

{'type_true': 6.213333333333333,
 'type_false': 7.400402414486922,
 'overall': 7.103635144198524}

In [27]:
avg_p_l_0 = 0
avg_p_l_1 = 0
for i in range(5):
    result = compute_avg_path_length_same_type(g, k=100)
    avg_p_l_0 += result["type_false"] / 2
    avg_p_l_1 += result["type_true"] / 2
# → {'type_true': 2.0, 'type_false': 0.0, 'overall': 1.3333}

print("T0:", avg_p_l_0/5, "   T1:", avg_p_l_1/5)

T0: 3.669019658284563    T1: 3.2922002635046113


In [24]:
g.density()

1.1241883410574385e-05

In [25]:
import statistics

statistics.mean(g.degree(g.vs.select(type=0)))

3.832770305494735

In [26]:
statistics.mean(g.degree(g.vs.select(type=1)))

11.503438348341065